In [2]:
import e2e_sae
import sys
from e2e_sae.scripts.analysis.utils import create_run_df, get_df_gpt2

In [3]:
df = get_df_gpt2()

Processing runs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 299/299 [00:10<00:00, 29.59it/s]


# Find Models that were used in the performance plot of the paper

In [5]:
performance_df = df.loc[(df["ratio"] == 60) & (df["seed"] == 0) & (df["n_samples"] == 400_000)]
# For layer 2 we filter out the runs with L0 > 200. Otherwise we end up with point in one
# subplot but not the other
performance_df = performance_df.loc[
    ~((performance_df["L0"] > 200) & (performance_df["layer"] == 2))
]
# Ignore specialised runs
performance_df = performance_df.loc[
    ~performance_df["name"].str.contains("seed-comparison")
    & ~performance_df["name"].str.contains("lr-comparison")
    & ~performance_df["name"].str.contains("lower-downstream")
    & -performance_df["name"].str.contains("e2e-local")
    & ~performance_df["name"].str.contains("recon-all")
    & ~performance_df["name"].str.contains("misc_")
]


In [8]:
performance_df["run_type"].unique()

array(['downstream', 'local', 'e2e'], dtype=object)

In [11]:
performance_df.columns

Index(['name', 'id', 'sae_pos', 'model_name', 'run_type', 'layer', 'seed',
       'n_samples', 'lr', 'ratio', 'sparsity_coeff', 'in_to_orig_coeff',
       'kl_coeff', 'out_to_in', 'L0', 'explained_var', 'explained_var_ln',
       'CE_diff', 'CELossIncrease', 'alive_dict_elements', 'mean_grad_norm',
       'explained_var_layer-11', 'explained_var_layer-7',
       'explained_var_layer-10', 'explained_var_layer-9',
       'explained_var_layer-8', 'explained_var_layer-6',
       'explained_var_ln_layer-7', 'explained_var_ln_layer-9',
       'explained_var_ln_layer-10', 'explained_var_ln_layer-11',
       'explained_var_ln_layer-8', 'explained_var_ln_layer-6',
       'recon_loss_layer-7', 'recon_loss_layer-10', 'recon_loss_layer-8',
       'recon_loss_layer-11', 'recon_loss_layer-9', 'recon_loss_layer-6',
       'sum_recon_loss', 'kl', 'explained_var_layer-2',
       'explained_var_ln_layer-2', 'recon_loss_layer-2',
       'explained_var_layer-1', 'explained_var_layer-3',
       'explained_

In [12]:
columns_list = ["id", "sae_pos", "run_type", "layer", "L0", "CELossIncrease", "alive_dict_elements"]

https://www.lesswrong.com/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream#Architecture_and_Hyperparameters

L0 is around 49.68 in Joseph Bloom's SAE models

In [14]:
performance_df[(performance_df["layer"] == 6) & (performance_df["run_type"] == "local")].sort_values("L0")[columns_list]

,id,sae_pos,run_type,layer,L0,CELossIncrease,alive_dict_elements
296,ahxwn90o,blocks.6.hook_resid_pre,local,6,10.271713,1.396334,3705
297,43zmudf4,blocks.6.hook_resid_pre,local,6,17.468240,0.675660,8785
285,unji5etq,blocks.6.hook_resid_pre,local,6,30.945175,0.326184,16690
295,jup3glm9,blocks.6.hook_resid_pre,local,6,42.798630,0.228030,20760
287,h9hrelni,blocks.6.hook_resid_pre,local,6,53.223964,0.184299,23480
298,1jy3m5j0,blocks.6.hook_resid_pre,local,6,69.426438,0.144315,26218
212,4nlqrc2y,blocks.6.hook_resid_pre,local,6,102.580427,0.109617,29503
213,2wvu1zs5,blocks.6.hook_resid_pre,local,6,183.948012,0.071096,33295
286,uiwt81f1,blocks.6.hook_resid_pre,local,6,512.987541,0.034243,36549


In [15]:
performance_df[(performance_df["layer"] == 6) & (performance_df["run_type"] == "e2e")].sort_values("L0")[columns_list]

,id,sae_pos,run_type,layer,L0,CELossIncrease,alive_dict_elements
251,4zcbb4au,blocks.6.hook_resid_pre,e2e,6,21.028400,0.354938,15955
250,zgdpkafo,blocks.6.hook_resid_pre,e2e,6,27.499313,0.144892,22295
249,tvj2owza,blocks.6.hook_resid_pre,e2e,6,39.681484,0.099144,24889
248,1bubkmps,blocks.6.hook_resid_pre,e2e,6,77.049683,0.056464,28510
247,wzzcimkj,blocks.6.hook_resid_pre,e2e,6,146.818863,0.035722,31871
246,4d5ksz89,blocks.6.hook_resid_pre,e2e,6,428.470720,0.017898,33624


In [16]:
performance_df[(performance_df["layer"] == 6) & (performance_df["run_type"] == "downstream")].sort_values("L0")[columns_list]

,id,sae_pos,run_type,layer,L0,CELossIncrease,alive_dict_elements
115,55get0e5,blocks.6.hook_resid_pre,downstream,6,31.445117,0.156221,12418
215,2lzle2f0,blocks.6.hook_resid_pre,downstream,6,36.789443,0.124463,14736
216,zidd0d3y,blocks.6.hook_resid_pre,downstream,6,46.487998,0.092411,17550
219,rb8q8czb,blocks.6.hook_resid_pre,downstream,6,58.823125,0.072060,20195
220,p9zmh62k,blocks.6.hook_resid_pre,downstream,6,90.038452,0.050133,23959
221,7nkdr21r,blocks.6.hook_resid_pre,downstream,6,131.283403,0.037811,27210
222,fqdgjxfe,blocks.6.hook_resid_pre,downstream,6,472.623281,0.015188,33842


In [24]:
list(performance_df[(performance_df["layer"] == 6) & (performance_df["run_type"] == "downstream")].sort_values("L0")[columns_list]["id"])

['55get0e5',
 '2lzle2f0',
 'zidd0d3y',
 'rb8q8czb',
 'p9zmh62k',
 '7nkdr21r',
 'fqdgjxfe']

In [20]:
df[df["id"] == "7nkdr21r"][columns_list]

,id,sae_pos,run_type,layer,L0,CELossIncrease,alive_dict_elements
221,7nkdr21r,blocks.6.hook_resid_pre,downstream,6,131.283403,0.037811,27210
